In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from tqdm import tqdm

In [2]:
data = pd.read_csv('datasets/Cat3.csv', sep=';')
data.head()

,Вопрос,Категория
0,Какое полномочие должно быть присвоено пользов...,Вопросы о регистрации и управлении учетными за...
1,Как пользователю присвоить полномочие «Региона...,Вопросы о регистрации и управлении учетными за...
2,Как присвоить полномочия пользователю компании...,Вопросы о регистрации и управлении учетными за...
3,Как перенести пользователя из одного профиля в...,Вопросы о регистрации и управлении учетными за...
4,Личный кабинет компании,Вопросы о регистрации и управлении учетными за...


In [3]:
'$'.join(i for i in data['Категория'].unique())

'Вопросы о регистрации и управлении учетными записями$Котировочные сессии и закупки$Ошибки сайта$Работа с контрактами$Справочная информация и поддержка$Стандартные товарные единицы (СТЕ)$Технические вопросы и настройки'

In [4]:
X = data['Вопрос']
y = data['Категория']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Используемое устройство: {device}')

Используемое устройство: cuda


In [6]:
train_data = Dataset.from_pandas(pd.DataFrame({'text': X_train, 'label': y_train.factorize()[0]}))
test_data = Dataset.from_pandas(pd.DataFrame({'text': X_test, 'label': y_test.factorize()[0]}))

In [7]:
# 'distilbert-base-multilingual-cased'
# 'bert-base-multilingual-cased'
# 'xlm-roberta-base'
# 'distilbert-base-uncased'
# 'roberta-base'
# 'albert-base-v2'

model_name = 'distilbert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(y_train.unique())).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_test = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/506 [00:00<?, ? examples/s]

Map:   0%|          | 0/127 [00:00<?, ? examples/s]

In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',
    logging_steps=10,
    report_to='none',
    fp16=torch.cuda.is_available()
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test
)

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.917900,2.466899
2,0.675300,2.585016
3,0.571800,2.651167


TrainOutput(global_step=96, training_loss=0.690874308347702, metrics={'train_runtime': 78.9235, 'train_samples_per_second': 19.234, 'train_steps_per_second': 1.216, 'total_flos': 201103441532928.0, 'train_loss': 0.690874308347702, 'epoch': 3.0})

In [13]:
trainer.save_model('./models/class.model')

In [14]:
model = AutoModelForSequenceClassification.from_pretrained('./models/class.model').to(device)

In [15]:
def classify_question_llm(question):
    inputs = tokenizer(question, return_tensors='pt', padding=True, truncation=True).to(device)
    outputs = model(**inputs)
    predicted_class = outputs.logits.argmax(dim=-1).item()
    return y_train.unique()[predicted_class]

In [16]:
data['Категория'].unique()

array(['Вопросы о регистрации и управлении учетными записями',
       'Котировочные сессии и закупки', 'Ошибки сайта',
       'Работа с контрактами', 'Справочная информация и поддержка',
       'Стандартные товарные единицы (СТЕ)',
       'Технические вопросы и настройки'], dtype=object)

In [17]:
example_question = 'Как изменить единицу измерения в оферте?'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Технические вопросы и настройки


In [18]:
example_question = 'Что такое портал поставщиков'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Справочная информация и поддержка


In [19]:
example_question = 'Требования к изображению'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Справочная информация и поддержка


In [20]:
example_question = 'ошибка РДИК'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Ошибки сайта


In [21]:
example_question = 'Заявка отклонена без объяснения причин'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Технические вопросы и настройки


In [22]:
example_question = 'У меня есть печка тандыр, что мне делать на портале поставщиков'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Технические вопросы и настройки


In [23]:
example_question = 'Хочу продавать в школы какое и кисель. Есть идеи?'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Справочная информация и поддержка


In [24]:
example_question = 'Хочу премиум аккаунт на портале, куда мне обратиться?'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Технические вопросы и настройки
